# Download all flower development related gene sequences
### Created 3 sets of gene-seqs:
* Set of genes from **3x-filtered** flower development related PMIDs: **108 genes** - saved in `refgenes/two_times_filtered`
* Set of genes from **2x-filtered** flower development related PMIDs: **165 genes** - saved in `refgenes/three_times_filtered`
* (Additional) Set of genes from **earlier transcriptomic papers**, where they were proved to be DEG in flower development related experiments: this list is not comparable to the other two, as the genes havent been proved to have a real flower-related function! It is only interesting to look at such lists and re-analyse them. The used papers: ..., The number of genes: **~170 genes** - saved in `refgenes/DEG_analyses_based`
## 1. Data import

In [2]:
library(taxize)
library(dplyr,  warn.conflicts = FALSE)
library(genbankr)
library(rentrez)
library(usethis)
library(parallel)
library(foreach, warn.conflicts = FALSE)
library(doParallel)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refpapers/")

Loading required package: iterators


In [3]:
# pmid-flower-list with 2x-overlap - 898 PMIDs
pmid_flower <- readRDS("PMIDlist_flower.rds")
pmid_flower <- as.integer(pmid_flower)
pmid_flower <- as.data.frame(pmid_flower, rownames = FALSE, stringsAsFactors = FALSE)
colnames(pmid_flower) <- "pmid"
glimpse(pmid_flower)
# pmid-flower-list with 3x-overlap - 308 PMIDs
pmid_flower_three <- readRDS("PMIDlist_flower_3.rds")
pmid_flower_three <- as.integer(pmid_flower_three)
pmid_flower_three <- as.data.frame(pmid_flower_three, rownames = FALSE, stringsAsFactors = FALSE)
colnames(pmid_flower_three) <- "pmid"
glimpse(pmid_flower_three)

Rows: 898
Columns: 1
$ pmid <int> 31963482, 31628536, 31473791, 31009129, 31729646, 31185903, 3051…
Rows: 308
Columns: 1
$ pmid <int> 29581960, 26157170, 24762030, 24162539, 30518316, 28163591, 2852…


In [4]:
# Downloaded Gene2Pubmed from ftp://ftp.ncbi.nih.gov/gene/DATA/gene2pubmed.gz @03.04.2020
gene2pm <- read.table("gene2pubmed", sep = "")
colnames(gene2pm) <- c("taxid", "geneid", "pmid")
glimpse(gene2pm)
head(gene2pm)

Rows: 12,465,900
Columns: 3
$ taxid  <int> 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 33, 33, 33, 33…
$ geneid <int> 1246500, 1246501, 1246502, 1246503, 1246504, 1246505, 1246509,…
$ pmid   <int> 9873079, 9873079, 9873079, 9873079, 9873079, 9873079, 10984505…


,taxid,geneid,pmid
,<int>,<int>,<int>
1,9,1246500,9873079
2,9,1246501,9873079
3,9,1246502,9873079
4,9,1246503,9873079
5,9,1246504,9873079
6,9,1246505,9873079


## 2. Get *Triticeae*-taxIDs into single vector

In [5]:
uid <- get_uid(sciname = "Triticeae")
uid[1]

══  1 queries  ═══════════════



Retrieving data for taxon 'Triticeae'



✔  Found:  Triticeae
══  Results  ═════════════════

● Total: 1 
● Found: 1 
● Not Found: 0


[1] "147389"

In [6]:
# 147389 is the id for Triticeae
tax_children <- downstream(uid[1], downto = "species", db = "ncbi")
glimpse(tax_children)

List of 1
 $ 147389:'data.frame':	389 obs. of  3 variables:
  ..$ childtaxa_id  : chr [1:389] "2259637" "1986981" "1986980" "1986370" ...
  ..$ childtaxa_name: chr [1:389] "Aegilops tauschii x Triticum aestivum" "Aegilops longissima x Triticum urartu" "Triticum urartu x Aegilops tauschii" "Triticum urartu x Aegilops longissima" ...
  ..$ rank          : chr [1:389] "species" "species" "species" "species" ...
 - attr(*, "class")= chr "downstream"
 - attr(*, "db")= chr "ncbi"


**For future problem solving:**
I had to put `ENTREZ_KEY='532056952c2098c0cd03a43bc25e345a7f08'` into my `.Renviron` file in the path of `/home/pgsb/vanda.marosi/` using the `usethis::edit_r_environ()` and terminal navigation with bash and restarted R

In [7]:
# convert to data.frame and data wrangling for tidy data
attributes(tax_children) <- NULL
tax <- as.data.frame(tax_children, stringsAsFactors = FALSE)
colnames(tax) <- c("taxid", "name", "rank")
glimpse(tax)

Rows: 389
Columns: 3
$ taxid <chr> "2259637", "1986981", "1986980", "1986370", "1897139", "1897138…
$ name  <chr> "Aegilops tauschii x Triticum aestivum", "Aegilops longissima x…
$ rank  <chr> "species", "species", "species", "species", "species", "species…


In [8]:
# total of 389 taxid-s found for triticeae
tax_id <- as.integer(tax$taxid)
taxid <- as.data.frame(tax_id, rownames = FALSE)
colnames(taxid) <- "taxid"
str(taxid)

'data.frame':	389 obs. of  1 variable:
 $ taxid: int  2259637 1986981 1986980 1986370 1897139 1897138 1593905 1593904 1245757 866788 ...


## 3. Intersect `tax_id` and `pmid_flower` with `Gene2Pubmed` table

In [9]:
#intersect taxid with gene2pubmed
trittax_geneid_pmid <- inner_join(gene2pm, taxid, by = "taxid")
glimpse(trittax_geneid_pmid)
# 27 930 geneIDs are linked to Triticeae taxid-s

Rows: 27,930
Columns: 3
$ taxid  <int> 4483, 4483, 4483, 4483, 4483, 4483, 4483, 4483, 4483, 4483, 44…
$ geneid <int> 20356298, 20356299, 20356300, 20356301, 20356302, 20356303, 20…
$ pmid   <int> 25059383, 25059383, 25059383, 25059383, 25059383, 25059383, 25…


In [10]:
# intersect pmid_flower with gene2pubmed that was already filtered with taxid-s to get only Triticeae relevant genes
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes")
# for 2x-overlap list 165 genes are found
trittax_geneid_flowerpmid <- inner_join(trittax_geneid_pmid, pmid_flower, by = "pmid")
write.table(trittax_geneid_flowerpmid, file = "triticeae_twoxfiltered_geneid_pmid.txt", append = FALSE, quote = FALSE, sep = "\t", dec = ".",
            row.names = FALSE, col.names = TRUE)

# save list for Uniport analyses
two <- select(trittax_geneid_flowerpmid, geneid)
glimpse(two)
write.table(two, file = "triticeae_two_genelist.txt", append = FALSE, quote = FALSE, sep = "\t", dec = ".",
            row.names = FALSE, col.names = TRUE)
glimpse(trittax_geneid_flowerpmid)

# for 3x-overlap list 108 genes are found - focusing on this list mainly!!
trittax_geneid_flowerpmid_three <- inner_join(trittax_geneid_pmid, pmid_flower_three, by = "pmid")

# save list for Uniport analyses
three <- select(trittax_geneid_flowerpmid_three, geneid)
glimpse(three)
write.table(three, file = "triticeae_three_genelist.txt", append = FALSE, quote = FALSE, sep = "\t", dec = ".",
            row.names = FALSE, col.names = TRUE)
glimpse(trittax_geneid_flowerpmid_three)

Rows: 165
Columns: 1
$ geneid <int> 548127, 548227, 732672, 732690, 100286750, 100498855, 10049885…
Rows: 165
Columns: 3
$ taxid  <int> 4513, 4513, 4513, 4513, 4513, 4513, 4513, 4513, 4513, 4513, 45…
$ geneid <int> 548127, 548227, 732672, 732690, 100286750, 100498855, 10049885…
$ pmid   <int> 24260147, 8982067, 22242122, 11925035, 19305410, 20431086, 204…
Rows: 108
Columns: 1
$ geneid <int> 100498855, 100498856, 542800, 543003, 543004, 543009, 543010, …
Rows: 108
Columns: 3
$ taxid  <int> 4513, 4513, 4565, 4565, 4565, 4565, 4565, 4565, 4565, 4565, 45…
$ geneid <int> 100498855, 100498856, 542800, 543003, 543004, 543009, 543010, …
$ pmid   <int> 20431086, 20431086, 14652757, 12061894, 12061894, 14612572, 15…


# 3.1 Inspect few example transcriptomics article whether it is present with genes in `Gene2pubmed` database/table
* title: Identification of Wheat Inflorescence Development-Related Genes Using a Comparative Transcriptomics Approach - PMID 29581960 
* title: Genome-Wide Investigation of Heat Shock Transcription Factor Family in Wheat (Triticum aestivum L.) and Possible Roles in Anther Development. - PMID 31963482

In [11]:
#trittax_geneid_flowerpmid$pmid[[29581960]]
#which(trittax_geneid_flowerpmid$pmid == 29581960)
#which(pmid_flower == 29581960)
which(gene2pm$pmid == 29581960)
which(gene2pm$pmid == 31963482)
# they are not present on the GeneID list

integer(0)

integer(0)

# 4. Download Geneseqs for flower-dev related geneIDs
### 4.1 `Rentrez`
* for batch download from `nuccore` database both in the formats of `rettype =`
    - **Genbank**:`gb` = GenBank flat file OR `gbwithparts` = GenBank flat file with full sequence (contigs)
    - **FASTA**: `fasta_cds_na` = CDS nucleotide FASTA OR `fasta_cds_aa` = CDS protein FASTA (for nucleotide I just used `fasta` bc the other did not give results for all the id-s)

### 4.1.1 `Three_times_filtered` flower-dev related list of genes

In [11]:
# create genelist from dataframe
genelist <- trittax_geneid_flowerpmid_three$geneid
tail(genelist)

[1] 100682416 100682417 100682418 100682419 101290634 109738555

In [12]:
# to fetch a single GeneID I have to link from GeneDataBase to Nuccore (works with different UID)
# within $link choose $gene_nuccore_refseqrna (for transcript) but most cases just have $gene_nuccore and the two numbers are equal in a few checked examples
# here is an example how to access id-conversion
gene_ids <- c(109738555)
linked_seq_ids <- entrez_link(dbfrom="gene", id=gene_ids, db="nuccore")
glimpse(linked_seq_ids)
linked_seq_ids$links

List of 2
 $ links:List of 3
  ..$ gene_nuccore          : chr [1:8] "1149780475" "1149780473" "1149780471" "1143453207" ...
  ..$ gene_nuccore_pos      : chr "1143453207"
  ..$ gene_nuccore_refseqrna: chr [1:3] "1149780475" "1149780473" "1149780471"
  ..- attr(*, "class")= chr [1:2] "elink_classic" "list"
 $ file :Classes 'XMLInternalElementNode', 'XMLInternalNode', 'XMLAbstractNode' <externalptr> 
 - attr(*, "content")= chr " $links: IDs for linked records from NCBI\n "
 - attr(*, "class")= chr [1:2] "elink" "list"


elink result with information from 3 databases:
[1] gene_nuccore           gene_nuccore_pos       gene_nuccore_refseqrna

In [13]:
nuccore_id_list <- foreach(i=genelist, .packages ="rentrez", .combine = 'c') %do% (entrez_link(dbfrom="gene", id = i, db="nuccore"))$links$gene_nuccore 
str(nuccore_id_list)

 chr [1:152] "298704707" "298704709" "669026884" "45356042" "660451103" ...


In [14]:
# loop for Genbank format
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/three_times_filtered/genbank")
cl <- makeCluster(3)
registerDoParallel(cl)
a <- foreach(i=nuccore_id_list, .packages="rentrez") %dopar% {
    seq <- entrez_fetch(db="nuccore", id=i, rettype="gbwithparts")
    write(seq, file=paste(i,".gb",sep=""))
    }
stopCluster(cl)

In [28]:
# differences between data-types
# nucleotide CDS
seq1 <- entrez_fetch(db="nuccore", id=298704707, rettype="fasta_cds_na")
class(seq1)
nchar(seq1)
cat(strwrap(substr(seq1, 1, 200)), sep="\n")
# protein CDS
seq2 <- entrez_fetch(db="nuccore", id=298704707, rettype="fasta_cds_aa")
nchar(seq2)
cat(strwrap(substr(seq2, 1, 200)), sep="\n")
# fasta
seq3 <- entrez_fetch(db="nuccore", id=298704707, rettype="fasta")
nchar(seq3)
cat(strwrap(substr(seq3, 1, 200)), sep="\n")
# GenBank flat file
seq4 <- entrez_fetch(db="nuccore", id=298704707, rettype="gb")
nchar(seq4)
cat(strwrap(substr(seq4, 1, 300)), sep="\n")
# GenBank flat file with full sequence
seq5 <- entrez_fetch(db="nuccore", id=298704707, rettype="gbwithparts")
nchar(seq5)
cat(strwrap(substr(seq5, 1, 300)), sep="\n")

[1] "character"

[1] 634

>lcl|HM130525.1_cds_ADI96237.1_1 [gene=ODDSOC1] [protein=ODDSOC1]
[protein_id=ADI96237.1] [location=30..533] [gbkey=CDS]
ATGGCGCGGCGCGGGCGGGTTGAGCTGCGGCGGATCGAGGACCGGACGAGCCGGCAGGTGCGCTTCTCCA
AGCGCCGC


[1] 293

>lcl|HM130525.1_prot_ADI96237.1_1 [gene=ODDSOC1] [protein=ODDSOC1]
[protein_id=ADI96237.1] [location=30..533] [gbkey=CDS]
MARRGRVELRRIEDRTSRQVRFSKRRAGLFKKAFELSLLCDAEVALLVFSPAGKLYEYSSASIEGTYDRY
QQFAVPG


...

>HM130525.1 Hordeum vulgare subsp. vulgare cultivar Igri ODDSOC1
(ODDSOC1) mRNA, complete cds
GAGAGACGGGCTGCGGCCGGAGGAGGAGGATGGCGCGGCGCGGGCGGGTTGAGCTGCGGCGGATCGAGGA
CCGGACGAGCCGGCAGGTGCGCTTCTCCAAGCGCC


[1] 3055

LOCUS HM130525 787 bp mRNA linear PLN 26-JUN-2010 DEFINITION Hordeum
vulgare subsp. vulgare cultivar Igri ODDSOC1 (ODDSOC1) mRNA, complete
cds. ACCESSION HM130525 VERSION HM130525.1 KEYWORDS . SOURCE Hordeum
vulgare subsp. vulgare (domesticate


[1] 3055

LOCUS HM130525 787 bp mRNA linear PLN 26-JUN-2010 DEFINITION Hordeum
vulgare subsp. vulgare cultivar Igri ODDSOC1 (ODDSOC1) mRNA, complete
cds. ACCESSION HM130525 VERSION HM130525.1 KEYWORDS . SOURCE Hordeum
vulgare subsp. vulgare (domesticate


In [16]:
# loop for fasta_nucleotide format into fasta (fasta_cds_na doesnt give results for all the id-s)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/three_times_filtered/fasta_nucleotide")
cl <- makeCluster(3)
registerDoParallel(cl)
b <- foreach(i=nuccore_id_list, .packages="rentrez") %dopar% {
    seq <- entrez_fetch(db="nuccore", id=i, rettype="fasta")
    write(seq, file=paste(i,".fasta",sep=""))
    }
stopCluster(cl)

In [17]:
# loop for fasta_protein format into - doesnt seem to work for all! empty ones need to be deleted in bash
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/three_times_filtered/fasta_protein")
cl <- makeCluster(3)
registerDoParallel(cl)
c <- foreach(i=nuccore_id_list, .packages="rentrez") %dopar% {
    seq <- entrez_fetch(db="nuccore", id=i, rettype="fasta_cds_aa")
    write(seq, file=paste(i,".fasta",sep=""))
    }
stopCluster(cl)

### 4.1.1 `Two_times_filtered` flower-dev related list of genes

In [13]:
# create genelist from dataframe
genelist_two <- trittax_geneid_flowerpmid$geneid
glimpse(genelist_two)
# convert to nuccore id-s
nuccore_id_list_two <- foreach(i=genelist_two, .packages ="rentrez", .combine = 'c') %do% (entrez_link(dbfrom="gene", id = i, db="nuccore"))$links$gene_nuccore 

setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered")
write.table(nuccore_id_list_two, file = "twotimes_filtered_accession.txt", append = FALSE, quote = FALSE, sep = "\t", dec = ".",
            row.names = FALSE, col.names = TRUE)
glimpse(nuccore_id_list_two)

 int [1:165] 548127 548227 732672 732690 100286750 100498855 100498856 100505452 100505453 100505454 ...
 chr [1:254] "13991760" "1808986" "255918289" "255918287" "59804999" ...


In [11]:
# loop for Genbank format
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered/genbank")
cl <- makeCluster(3)
registerDoParallel(cl)
a <- foreach(i=nuccore_id_list_two, .packages="rentrez") %dopar% {
    seq <- entrez_fetch(db="nuccore", id=i, rettype="gbwithparts")
    write(seq, file=paste(i,".gb",sep=""))
    }
stopCluster(cl)

In [14]:
# loop for fasta_nucleotide format into fasta (fasta_cds_na doesnt give results for all the id-s)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered/fasta_nucleotide")
cl <- makeCluster(3)
registerDoParallel(cl)
b <- foreach(i=nuccore_id_list_two, .packages="rentrez") %dopar% {
    seq <- entrez_fetch(db="nuccore", id=i, rettype="fasta")
    write(seq, file=paste(i,".fasta",sep=""))
    }
stopCluster(cl)

In [15]:
#  loop for fasta_protein format into - doesnt seem to work for all! empty ones need to be deleted in bash
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered/fasta_protein")
cl <- makeCluster(3)
registerDoParallel(cl)
c <- foreach(i=nuccore_id_list_two, .packages="rentrez") %dopar% {
    seq <- entrez_fetch(db="nuccore", id=i, rettype="fasta_cds_aa")
    write(seq, file=paste(i,".fasta",sep=""))
    }
stopCluster(cl)

### 4.1.1 Additionally collected flower-dev related list of genes

Daniel sent one, + the first paper he sent, + the papers that came from transcriptomics filter but were experimental

would be interesting to create a set as "false negatives" but after finishing the rest of the analysis

# 5. Mapping
## 5.1 Parsing sequence files with `Genbank-r`
* a package for parsing GenBank files into semantically useful objects, able to read Genbank files
* `readGenBank()`: takes either local file or text in a form of a GBAccession object containing Nuccore versioned accession numbers
    - uses `rentrez`
    - generates a GenBankFull object, which contains the annotations and the origin sequence
    - useful tutorial: https://bioconductor.org/packages/release/bioc/vignettes/genbankr/inst/doc/genbankr.html

In [22]:
Sys.getenv("ENTREZ_KEY")

[1] "532056952c2098c0cd03a43bc25e345a7f08"

In [54]:
gb_names <- list.files(path="/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered/genbank", full.names=FALSE)
str(gb_names)

 chr [1:242] "1027405096.gb" "1036031969.gb" "108795022.gb" "109150347.gb" ...


In [55]:
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered/genbank")
suppressPackageStartupMessages(library(genbankr))
#here is a package default example
#smpfile = system.file("sample.gbk", package="genbankr")
#here is one of my files
gb = parseGenBank("1027405096.gb")

In [4]:
str(gb)

List of 11
 $ LOCUS     : chr "LOCUS       IAAV01057863             252 bp    RNA     linear   TSA 11-MAY-2016"
 $ FEATURES  :List of 1
  ..$ 1:'data.frame':	1 obs. of  13 variables:
  .. ..$ seqnames   : chr "KU-2627"
  .. ..$ start      : int 1
  .. ..$ end        : int 252
  .. ..$ strand     : chr "+"
  .. ..$ type       : chr "source"
  .. ..$ organism   : chr "Aegilops tauschii"
  .. ..$ mol_type   : chr "transcribed RNA"
  .. ..$ strain     : chr "KU-2627"
  .. ..$ db_xref    : chr "taxon:37682"
  .. ..$ tissue_type: chr "leaves"
  .. ..$ dev_stage  : chr "ten-day-old seedling"
  .. ..$ note       : chr "contig: KU2627_c42261_g1_i1"
  .. ..$ loctype    : chr "normal"
  ..- attr(*, "dim")= int 1
  ..- attr(*, "dimnames")=List of 1
  .. ..$ : chr "1"
 $ ORIGIN    :Formal class 'DNAStringSet' [package "Biostrings"] with 5 slots
  .. ..@ pool           :Formal class 'SharedRaw_Pool' [package "XVector"] with 2 slots
  .. .. .. ..@ xp_list                    :List of 1
  .. .. .. .. .

In [5]:
#extract information for grouping files: organism, mol_type
a <- gb$FEATURES
a <- unlist(a)
a[[6]]
a[[7]]

[1] "Aegilops tauschii"

[1] "transcribed RNA"

In [56]:
#extract metadata into vector
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered/genbank")
gb_organism <- foreach(i=gb_names, .packages ="genbankr", .combine = "c") %do% (suppressWarnings(unlist(parseGenBank(i)$FEATURES))[[6]])
gb_moltype <- foreach(i=gb_names, .packages ="genbankr", .combine = "c") %do% (suppressWarnings(unlist(parseGenBank(i)$FEATURES))[[7]])
glimpse(gb_organism)
glimpse(gb_moltype)

Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound range (join or order) feature. Excluding entire annotation
Partial range detected in a compound ran

 chr [1:242] "Aegilops tauschii" "Triticum aestivum" "Triticum aestivum" ...
 chr [1:242] "transcribed RNA" "genomic DNA" "mRNA" "mRNA" "mRNA" "mRNA" ...


In [57]:
gb_names <- as.data.frame(gb_names, stringsAsFactors = FALSE)
gb_organism <- as.data.frame(gb_organism, stringsAsFactors = FALSE)
gb_moltype <- as.data.frame(gb_moltype, stringsAsFactors = FALSE)
gb_meta <- bind_cols(gb_names, gb_organism, gb_moltype)
colnames(gb_meta) <- c("filename", "organism", "molecule")
glimpse(gb_meta)

Rows: 242
Columns: 3
$ filename <chr> "1027405096.gb", "1036031969.gb", "108795022.gb", "109150347…
$ organism <chr> "Aegilops tauschii", "Triticum aestivum", "Triticum aestivum…
$ molecule <chr> "transcribed RNA", "genomic DNA", "mRNA", "mRNA", "mRNA", "m…


### Separation of fasta_nucleotide files based on organism & mol_type

In [58]:
library(stringr)
# grouping by organism
triticum <- dplyr::filter(gb_meta, organism == "Triticum aestivum")
glimpse(triticum)
hordeum <- dplyr::filter(gb_meta, organism %in% c("Hordeum vulgare","Hordeum vulgare subsp. vulgare"))
glimpse(hordeum)
aegilops <- dplyr::filter(gb_meta, organism %in% c("Aegilops tauschii","Aegilops tauschii subsp. tauschii"))
glimpse(aegilops)

Rows: 219
Columns: 3
$ filename <chr> "1036031969.gb", "108795022.gb", "109150347.gb", "109150355.…
$ organism <chr> "Triticum aestivum", "Triticum aestivum", "Triticum aestivum…
$ molecule <chr> "genomic DNA", "mRNA", "mRNA", "mRNA", "mRNA", "genomic DNA"…
Rows: 15
Columns: 3
$ filename <chr> "13991760.gb", "151419595.gb", "1808986.gb", "208293841.gb",…
$ organism <chr> "Hordeum vulgare", "Hordeum vulgare subsp. vulgare", "Hordeu…
$ molecule <chr> "mRNA", "mRNA", "mRNA", "mRNA", "genomic DNA", "genomic DNA"…
Rows: 8
Columns: 3
$ filename <chr> "1027405096.gb", "1143453207.gb", "1149780471.gb", "11497804…
$ organism <chr> "Aegilops tauschii", "Aegilops tauschii subsp. tauschii", "A…
$ molecule <chr> "transcribed RNA", "genomic DNA", "mRNA", "mRNA", "mRNA", "g…


In [75]:
# group triticum by molecule
# nosplicing
t_nosplicing <- dplyr::filter(triticum, molecule %in% c("genomic DNA"))
glimpse(t_nosplicing)
# save filename-list
t_nosplicing_filename <- pull(t_nosplicing, filename)
t_nosplicing_filename <- str_replace(t_nosplicing_filename, ".gb", ".fasta")
glimpse(t_nosplicing_filename)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/triticum/")
write.table(t_nosplicing_filename, file = "wheat_nosplicing.csv", append = FALSE, quote = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)

# splicing
t_splicing <- dplyr::filter(triticum, molecule %in% c("unassigned DNA", "transcribed RNA", "mRNA"))
glimpse(t_splicing)
# save filename-list
t_splicing_filename <- pull(t_splicing, filename)
t_splicing_filename <- str_replace(t_splicing_filename, ".gb", ".fasta")
glimpse(t_splicing_filename)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/triticum/")
write.table(t_splicing_filename, file = "wheat_splicing.csv", append = FALSE, quote = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)

Rows: 46
Columns: 3
$ filename <chr> "1036031969.gb", "109240245.gb", "109450933.gb", "117168399.…
$ organism <chr> "Triticum aestivum", "Triticum aestivum", "Triticum aestivum…
$ molecule <chr> "genomic DNA", "genomic DNA", "genomic DNA", "genomic DNA", …
 chr [1:46] "1036031969.fasta" "109240245.fasta" "109450933.fasta" ...
Rows: 173
Columns: 3
$ filename <chr> "108795022.gb", "109150347.gb", "109150355.gb", "109150357.g…
$ organism <chr> "Triticum aestivum", "Triticum aestivum", "Triticum aestivum…
$ molecule <chr> "mRNA", "mRNA", "mRNA", "mRNA", "mRNA", "mRNA", "mRNA", "mRN…
 chr [1:173] "108795022.fasta" "109150347.fasta" "109150355.fasta" ...


In [76]:
# group hordeum by molecule
h_nosplicing <- dplyr::filter(hordeum, molecule %in% c("genomic DNA"))
glimpse(h_nosplicing)
# save filename-list
h_nosplicing_filename <- pull(h_nosplicing, filename)
h_nosplicing_filename <- str_replace(h_nosplicing_filename, ".gb", ".fasta")
glimpse(h_nosplicing_filename)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/hordeum/")
write.table(h_nosplicing_filename, file = "barley_nosplicing.csv", append = FALSE, quote = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)

h_splicing <- dplyr::filter(hordeum, molecule %in% c("unassigned DNA", "transcribed RNA", "mRNA"))
glimpse(h_splicing)
# save
h_splicing_filename <- pull(h_splicing, filename)
h_splicing_filename <- str_replace(h_splicing_filename, ".gb", ".fasta")
glimpse(h_splicing_filename)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/hordeum/")
write.table(h_splicing_filename, file = "barley_splicing.csv", append = FALSE, quote = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)

Rows: 6
Columns: 3
$ filename <chr> "255918287.gb", "255918289.gb", "308211036.gb", "308211038.g…
$ organism <chr> "Hordeum vulgare", "Hordeum vulgare", "Hordeum vulgare", "Ho…
$ molecule <chr> "genomic DNA", "genomic DNA", "genomic DNA", "genomic DNA", …
 chr [1:6] "255918287.fasta" "255918289.fasta" "308211036.fasta" ...
Rows: 9
Columns: 3
$ filename <chr> "13991760.gb", "151419595.gb", "1808986.gb", "208293841.gb",…
$ organism <chr> "Hordeum vulgare", "Hordeum vulgare subsp. vulgare", "Hordeu…
$ molecule <chr> "mRNA", "mRNA", "mRNA", "mRNA", "mRNA", "mRNA", "mRNA", "mRN…
 chr [1:9] "13991760.fasta" "151419595.fasta" "1808986.fasta" ...


In [77]:
# group aegilops by molecule
a_nosplicing <- dplyr::filter(aegilops, molecule %in% c("genomic DNA"))
glimpse(a_nosplicing)
# save filename-list
a_nosplicing_filename <- pull(a_nosplicing, filename)
a_nosplicing_filename <- str_replace(a_nosplicing_filename, ".gb", ".fasta")
glimpse(a_nosplicing_filename)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/aegilops/")
write.table(a_nosplicing_filename, file = "aegilops_nosplicing.csv", append = FALSE, quote = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)

a_splicing <- dplyr::filter(aegilops, molecule %in% c("unassigned DNA", "transcribed RNA", "mRNA"))
glimpse(a_splicing)
# save filename-list
a_splicing_filename <- pull(a_splicing, filename)
a_splicing_filename <- str_replace(a_splicing_filename, ".gb", ".fasta")
glimpse(a_splicing_filename)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/aegilops/")
write.table(a_splicing_filename, file = "aegilops_splicing.csv", append = FALSE, quote = FALSE, sep = " ", dec = ".",
            row.names = FALSE, col.names = FALSE)

Rows: 2
Columns: 3
$ filename <chr> "1143453207.gb", "343424461.gb"
$ organism <chr> "Aegilops tauschii subsp. tauschii", "Aegilops tauschii"
$ molecule <chr> "genomic DNA", "genomic DNA"
 chr [1:2] "1143453207.fasta" "343424461.fasta"
Rows: 6
Columns: 3
$ filename <chr> "1027405096.gb", "1149780471.gb", "1149780473.gb", "11497804…
$ organism <chr> "Aegilops tauschii", "Aegilops tauschii subsp. tauschii", "A…
$ molecule <chr> "transcribed RNA", "mRNA", "mRNA", "mRNA", "mRNA", "mRNA"
 chr [1:6] "1027405096.fasta" "1149780471.fasta" "1149780473.fasta" ...


Above files had to be concatenated into single `.fasta` files, despite trying to solve it in R (proof below) it turned to be better to use bash:
* `cat $(cat wheat_splicing.csv) > wheat_splicing.fasta` -the source table had to be space separated, command has to be executed in the directory of the `.fasta` files

In [60]:
# concatenate triticum nucleotide_fasta files into single file and save into gmap/triticum folder
### nosplicing
library(seqinr)
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/two_times_filtered/fasta_nucleotide")
t_nosplicing_cat <- foreach(i=t_nosplicing_filename, .packages ="seqinr") %do% read.fasta(file = i,  as.string = TRUE, forceDNAtolower = FALSE, whole.header = TRUE, set.attributes = FALSE)

In [26]:
head(t_nosplicing_cat)

[[1]]
[[1]]$`AH013429.2 Triticum aestivum cultivar Xinkehan 9 HMW glutenin subunit gene, complete cds`
[1] "GCCAGGGAAAGACAATGGACATGCAGAGAGGCGGGGGCGGGGAAGAAACACATGGAGATCATAGAAGAACATAAGAGGTTAAACATAGGAGGAGGATATAATGGACAATTAAATCCACATTACCTGAACTCATTTGGGAAGTGGAAAAATCCCCTATTCTGGTGTAAATCAAACTAATTGACGCGAGTTTTCTCTGAAGATTCTATGTTAATTTTAGACATGAATGACCAAAGGTTTCAGTTAGTTGAGTTTTGTCATCGAAAGGTGTTTACATAAGTCCAAAAATTCTACCAGCTTTTGGTACGGCGCGTCATAGAACAGATAAATGTTGTGAGTCATTGGATAGATATTATGAGTCATAGCATGGGTTTGTGTAGCCTGGAAATCTAACTATGACAAGAAACAAAACATAAATGGGCTTTTGAAAGATGATTTATCAACTTACCTTATCCATGCAAGCTACCTTCCACTAGTCGACATGCTTAGAAGCTTTTAGTGACCGCAGTATTTGCAAAAGCAATGGCTAACAGACACCCAAACCCCAAGAAGCATAACCACTTCTCTTAGATAAAAATAGCAGATCGATATACAAACGTCTACACTTCTGCAAACAATACCCAGAAGCCAGAATTAGGATTGAACCGATTACGTGGCTTTAGCAGACCGTCCAAAAATCTGTTTTGCAAAGCTCCAATTGCTCCTTGCTTATCCAGCTTCTTTTGTGTTGGCAAATTGTTCTTTTCCAACCGACTTTATTCTTTTCACATTTCTTCTTAGGCTGAACTAACCTTCGTCGTGCACACAACCATTGTCCTGAACCTTCACCACGTCCCTATAAAAGCCCAACCAATCTCCACAATTTCATCATCACCCACAACACCGAGCACCACAAAATAGAGATCAATTCACTGACAGTCCANNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNATGGCTAAGCGGCTGGTCCTCTTTGCGGCAGTAGTCATCGCCCTCGTGGCTCTCACCACCGCTGAAGGTGAGGCCTCTAGGCAACTACAGTGTGAGCGCGAGCTCCAGGAGAGCTCGCTTGAGGCATGCCGGCAGGTCGTGGACCAACAGTTGGCCGGTCGGCTGCCATGGAGCACGGGGCTCCAGATGCGATGCTGCCAGCAGCTCCGAGATGTTAGCGCCAAGTGCCGCTCTGTCGCCGTCAGCCAAGTCGCAAGACAATATGAGCAAACTGTGGTGCCGCCCAAGGGCGGATCCTTCTACCCTGGTGAGACCACGCCACTGCAGCAACTCCAACAAGGAATATTTTGGGGAACATCTTCACAAACAGTACAAGGGTATTACCCAAGCGTAACTTCTCCTCGGCAGGGGTCATATTATCCAGGCCAAGCTTCTCCACAACAGCCAGGACAAGGGCAACAGCCTGGCAAATGGCAAGAACCAGGACAAGGGCAACAATGGTACTACCCAACTTCTCTGCAGCAGCCAGGACAAGGGCAACAGATAGGAAAAGGGAAACAAGGGTACTACCCAACTTCTCTGCAGCAACCAGGACAAGGGCAACAAATAGGACAAGGGCAACAAGGGTACTACCCAACTTCTCCGCAGCACACAGGACAAAGGCAACAACCAGTACAAGGGCAACAAATAGGACAAGGGCAACAACCAGAACAAGGGCAACAACCAGGACAATGGCAACAAGGGTACTATCCAACTTCTCCACAGCAGCTAGGACAAGGGCAACAACCAGGACAATGGCAACAATCAGGACAAGGGCAACAAGGGCACTACCCAACTTCTCTACAACAGCCAGGACAAGGGCAACAAGGGCATTACCTAGCTTCTCAGCAGCAGCCAGCACAAGGGCAACAAGGGCACTACCCAGCTTCTCAGCAGCAGCCAGGACAAGGGCAACAAGGGCACTACCCAGCTTCTCAGCAGCAGCCAGGACAAGGGCAACAAGGGCACTACCCAGCTTCTCAGCAAGAGCCAGGACAAGGGCAACAAGGGCAAATCCCAGCTTCTCAGCAGCAGCCAGGACAAGGGCAACAAGGGCACTACCCAGCTTCTCTGCAGCAACCAGGACAACAAGGGCATTACCCAACTTCTCTACAGCAGCTAGGACAAGGGCAACAAATAGGACAGCCAGGACAAAAGCAACAACCAGGACAAGGGCAACAAACAGGACAAGGGCAACAGCCAGAACAAGAGCAACAACCAGGACAAGGGCAACAAGGATACTATCCAACTTCTCTGCAGCAGCCAGGACAAGGACAACAGCAAGGACAAGGGCAACAAGGGTACTACCCAACTTCTCTCCAGCAGCCAGGACAAGGGCAACAAGGGCACTACCCAGCTTCTCTGCAGCAGCCAGGACAAGGACAGCCAGGACAAAGGCAACAACCAGGACAAGGGCAACATCCAGAACAAGGTCAACAACCAGGACAAGGGCAACAAGGGTACTATCCAACTTCTCCACAGCAGCCAGGACAAGGGCAACAACTAGGACAAGGGCAACAAGGGTACTACCCAACTTCTCCGCAGCAGCCAGGACAAGGGCAACAACCAGGACAAGGGCAACAAGGGCACTGCCCAATGTCCCCGCAGCAGACAGGACAAGCGCAACAACTAGGACAAGGCCAACAAATAGGACAAGTGCAACAACCAGGACAAGGGCAACAAGGGTACTACCCAACTTCTCTGCAGCAGCCTGGACAAGGGCAACAGTCAGGACAAGGGCAACAGTCAGGACAAGGACACCAACCAGGACAAGGGCAGCAATCAGGACAAGAGAAACAAGGCTACGACAGCCCATACCATGTTAGCGCAGAGCAGCAAGCGGCCAGCCCAATGGTGGCAAAGGCGCAGCAGCCCGCGACACAGCTGCCGACAGTGTGTCGGATGGAGGGGGGCGACGCATTGTCGGCTAGCCAGTGATAG"


[[2]]
[[2]]$`DQ534491.1 Triticum aestivum MADS3 mRNA, complete cds`
[1] "AAGAAGAGAGAGGTAGAGAGAGGCCGGGGAAGAAGAGAGCCAGCGAGCGGCAAGGCGGCGGAAGGATGGGGCGCGGCAAGGTGGAGCTGAAGCGGATCGACAACAAGATCAGCCGGCAGGTGACGTTCGCCAAGCGCCGCAACGGGCTGCTCAAGAAGGCGTACGAGCTGTCGGTGCTCTGCGACGCCGAGGTCGCGCTCATCATCTTCTCCACCCGCGGCCGCCTCTTCGAGTTCTCCACATCCTCATGCATGTACAAGACACTAGAGCGGTACCGCAGCTGCAACTTCAACTCCGAGGCAACTGCAACTCCGGAGACTGAACAAAGCAATTACCAGGAGTACTTGAAGCTGAAGACAAGAGTTGAGTTCCTACAGACAACTCAGAGAAATCTTCTTGGCGAGGACTTGGGCCCACTTAACATGAAGGAACTTGAGCAGCTTGAGAACCAAATTGAGATATCTCTCAAACATATCAGGGCGACAAAGAGCCAACAGTCACTTGATCAGCTCTTTGAGCTCAAGCGCAAGGAACAACAACTGCAAGATGTTAATAAAGACTTACGGAAGAAGATACAAGAAACTAGTGTGGAAAATGTGCTGCAAATGTCTTGCCAGGACGTTGGACCTAGTGGGTCTAGTGGCCATGCTAATCAAGCTAATCAACAGGAGTATTTTCATCATGATTGTGACCCTTCCCTGCATATAGGGTATCAACGAAATTTCCTGGATCAGTTGAACAAAGAATGAATGGTTGTATGAGCTGAGCCTGAGCG"


[[3]]
[[3]]$`AB244637.1 Triticum aest

In [61]:
# DOES NOT WORK!!!! - fasta headers are cut off no matter what options are used
# write into file
setwd("/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/triticum")
#triticum_nosplicing <- tempfile(fileext = "fasta")
#tmpf <- tempfile()
write.fasta(sequences = t_nosplicing_cat, names = names(t_nosplicing_cat), file.out = "./triticum_nosplicing1.fasta", open = "w", nbchar = 60, as.string = TRUE, whole.header = TRUE)
test <- read.fasta("./triticum_nosplicing.fasta", set.attributes = TRUE)
identical(test, t_nosplicing_cat)

ERROR: Error in write.fasta(sequences = t_nosplicing_cat, names = names(t_nosplicing_cat), : unused argument (whole.header = TRUE)


## 4.3 Mapping with GMAP
* **official website:** http://research-pub.gene.com/gmap/
* my gmap is installed into `seqtools` conda environment
* due to the large genome size in barley&wheat I ahd to use `gmapl` instead, but it was installed along with gmap
* **perl scripts to inspect files:** 
    - `perl -ne 'print $1,"\n" if /mol_type="([^"]+)"/;' genbank/*.gb| sort | uniq -c`
    - `perl -ne 'print $1,"\n" if /^\s*ORGANISM\s+(.+)/;' genbank/*.gb| sort | uniq -c`
* **preparations:** 
    - gene-seqs belong to 3 organisms: triticum, hordeum, aegilops
    - they have 4 types of data: 
        - genomic DNA - `--nosplicing` flag has to be on! (Turns off splicing (useful for aligning genomic sequences onto a genome)
        - mRNA & transcribedRNA & unassigned (which we assign to transcribedRNA) - `--nosplicing` flag has to be off!
    - create dataframe with "Organism" and "mol_type" information to parse files
    - create concatenated merged fasta file from all the nucleotide gene fastas for each group to map (script and output paths are above)
    - create symlinks of each ref.genome into gmap folder for indexing
* **1.:** indexing
    - directory: `/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/indexes/gmap`
    - run indexing command in a screen session in the directory of the respective genome where symlinks are:
        
        `gmap_build -D /nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/indexes/gmap -d triticum -k 15 genome.fasta`
        
        `gmap_build -D /nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/indexes/gmap -d hordeum -k 15 genome.fasta`
        
        `gmap_build -D /nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/indexes/gmap -d aegilops -k 15 genome.fasta`
    - use the `-D` flag to specify the default location for the tool
    - it can take upto a day long
* **2.:** use Aegilops 2+1 fastas to create 12 different output formats
    - inspect result files and choose the option for the entire analysis
    - for splicing category only a single file was tested
    - for nosplicing the cat csv based 2 files
* **3.:** run the respective `gmap_gff.sh` script on slurm
    - here `-f` specifies the output file format, where `est.gff` and `alignment.gff` is the most recommended and can be later converted into counttable using STAR/HISAT
    - destination directories: `/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/refgenes/gmap/`
    - scripts are found: `home/pgsb/vanda.marosi/scripts/gmap/`

## Location of Ref.Genomes
* all of them are symlinked under: `/nfs/pgsb/projects/comparative_triticeae/phenotype/flower_development/genomes/`

`./Horvu/CDS.fasta:49281
./Horvu/protein.fasta:46294
./Horvu/transcript.fasta:49281
./Horvu/genome.fasta:8
./Triae/protein.fasta:122722
./Triae/genome.fasta:22
./Triae/transcript.fasta:123075
./Triae/CDS.fasta:122722
./Aegta/transcript.fasta:68789
./Aegta/genome.fasta:8
./Aegta/CDS.fasta:68789
./Aegta/protein.fasta:68390`

## 4.4 Filtering with identity&coverage values and intersection with ref.annotations

In [27]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /home/vanda.marosi/anaconda3/envs/r/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=de_DE.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=de_DE.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=de_DE.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=de_DE.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] doParallel_1.0.14 iterators_1.0.12  foreach_1.5.0     usethis_1.6.0    
[5] rentrez_1.2.2     genbankr_1.14.0   dplyr_0.8.5       taxize_0.9.94    

loaded via a namespace (and not attached):
 [1] nlme_3.1-147                bitops_1.0-6               
 [3] matrixStats_0.56.0   